In [14]:
from tkinter import *
from tkinter import messagebox
import pandas as pd 
from tkinter.filedialog import askopenfile
import tkinter as tk
from tkinter import filedialog


import numpy as np
import pandas as pd
import re
import nltk,warnings,gensim,re
from string import punctuation
nltk.download(['punkt','stopwords','wordnet','averaged_perceptron_tagger'])
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model, Sequential,load_model
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D, BatchNormalization,SimpleRNN,GRU
import tensorflow as tf
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\minak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\minak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\minak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\minak\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Milestone 1 & 2

In [15]:
window = tk.Tk()
window.title("FCR template")
var = IntVar()
model_status='Not Ready'
train = "Untrained"

#Pandas profiling process
def showMsg():
    try:
        if source_path and target_path:
            heart_data = pd.read_csv(source_path) 
            heart_data.head()
            report = ProfileReport(heart_data, title = "Sample Report")
            report.to_file(target_path+"/report_file.html")
            entry3.configure(text="Successfull!!",bg='#1fde72',font='Helvetica 8 bold')
            print(source_path)
    except:
        entry3.configure(text="Select Source file/Target directory",bg='#f03e3e',font='Helvetica 8 bold')
    
#Loading Source file    
def open_file():
    global source_path
    source_path = filedialog.askopenfile()   
    entry1.configure(text="File selected",bg='#1fde72',font='Helvetica 8 bold')
    entry4.configure(text="File selected",bg='#1fde72',font='Helvetica 8 bold')
    lb2.configure(text=source_path.name)
    lb7.configure(text=source_path.name)
    entry5.configure(text ='Generate Vectors',bg='#FFFF00',font='Helvetica 8 bold')
    if target_path:
        pass
    else:
        entry2.configure(text ='Click here for destination directory!!',bg='#FFFF00',font='Helvetica 8 bold')
    
#selecting Target directory    
def save_file():
    global target_path
    target_path = filedialog.askdirectory()   
    entry2.configure(text="Path selected",bg='#1fde72',font='Helvetica 8 bold')
    lb4.configure(text=target_path)   
    entry3.configure(text ='Generate EDA Report',bg='#FFFF00',font='Helvetica 8 bold')

#Function to concatenate    
def data(x,y,z,a):
    return x+" "+y+" "+z+" "+a

def remove_stopwords(x):
    x = re.sub(r"[\u200bâ€°]+","",x.lower())
    return " ".join(i for i in nltk.word_tokenize("".join([i for i in x if i not in punctuation ])) if i not in stopwords.words('english'))

def lemmatized(x):
    lm=WordNetLemmatizer()
    return lm.lemmatize(x)   

def W2V(x):
    w2v_l2 = Word2Vec(sentences=x,vector_size=100,min_count=1)
    return w2v_l2

def NN():
    global model_6
    model_6 = Sequential()
    model_6.add(Embedding(input_dim = len(TK1.word_index)+1,output_dim = 100, weights=[w2v_vector_s_c],input_length = 102))
    model_6.add(Flatten())
    model_6.add(Dense(500,activation='relu'))
    model_6.add(Dropout(0.5))
    model_6.add(Dense(250,activation='relu'))
    model_6.add(Dropout(0.5))
    model_6.add(Dense(120,activation='relu'))
    model_6.add(Dropout(0.5))
    model_6.add(Dense(5,activation='softmax'))
    model_6.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
    return model_6
    
def Train():
    global model_6,train
    if model_status=='Ready':
        ES = EarlyStopping(monitor='val_loss', min_delta=0.01,patience=10,mode='min')
        model_6.fit(X_train,y_train, epochs=100,validation_data=(X_test,y_test),callbacks=[ES])
        entry6.configure(text="Model Status : Trained!!!",bg='#1fde72',font='Helvetica 8 bold')
        train = "Trained"
    else:
        entry6.configure(text="Model Status : Not Ready",bg='#f03e3e',font='Helvetica 8 bold')  
        
def Test():
    if train == "Trained":
        prediction = model_6.predict(X_test)
        value,classes_pred = tf.math.top_k(prediction, k=1, sorted=True, name=None)
        classes_actual = y_test.idxmax(axis=1)
        x=classification_report(classes_actual,classes_pred)
        t.configure(text = classification_report(classes_actual,classes_pred))
    else:
        entry7.configure(text="Train the model",bg='#f03e3e',font='Helvetica 8 bold')  
    
    
        

def Preprocessing():
    global df,data_list_lemmatized_o,TK1,w2v_vector_s_c,model_status,X_train,X_test,y_train,y_test
    try:
        if source_path:
            df = pd.read_csv(source_path)
            df['Description_1'] = df.apply(lambda x: data(x['Industry Sector'],x['Employee or Third Party'],x['Critical Risk'],x['Description']),axis=1)
            df['Description_1'] = df['Description_1'].apply(remove_stopwords)
            df['Description_1lemmatized'] = df['Description_1'].apply(lemmatized)
            data_list_lemmatized_o = []
            for i in df['Description_1lemmatized']:
                data_list_lemmatized_o.append(nltk.word_tokenize(i))

            w2v = W2V(data_list_lemmatized_o)

            LE1 = LabelEncoder()
            classes_actual = LE1.fit_transform(df['Accident Level'])
            y = pd.get_dummies(classes_actual)

            TK1 = Tokenizer()
            TK1.fit_on_texts(df['Description_1lemmatized'])

            w2v_vector_s_c = np.zeros((len(TK1.word_index)+1, 100))
            for word, i in TK1.word_index.items():
                embedding_vector = w2v.wv.get_vector(word)
                if embedding_vector is not None:
                    w2v_vector_s_c[i] = embedding_vector

            data_1 = TK1.texts_to_sequences(df['Description_1lemmatized'])

            X = pad_sequences(data_1,maxlen=102)
            y = pd.get_dummies(classes_actual)

            X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=101)
            model = NN()
            entry6.configure(text="Model Status : Ready to Train!!!",bg='#1fde72',font='Helvetica 8 bold')
            model_status='Ready'


        else:
            entry4.configure(text="Select Source file",bg='#f03e3e',font='Helvetica 8 bold')            
    except:
        entry4.configure(text="Select Source file",bg='#f03e3e',font='Helvetica 8 bold')
        
    
    
#Labels, buttons and entries - Milestone1
lb = tk.Label(window, text="Milestone 1 - EDA",font='Helvetica 8 bold')
lb.grid(column=1, row=0)    
llb = tk.Label(window, text="")
llb.grid(column=0, row=1)

lb1 = tk.Label(window, text="Step1: Select Source file",font='Helvetica 8 bold')
lb1.grid(column=0, row=5)
entry1 = tk.Button(window,width=30, text ='Click here to load file!!', command = lambda:open_file(),font='Helvetica 8 bold')
entry1.grid(column=1, row=5)    
lb2 = tk.Label(window, text="                                                                                           ")
lb2.grid(column=2, row=5)

lb3 = tk.Label(window, text="Step2: Select Destination location",font='Helvetica 8 bold')
lb3.grid(column=0, row=7)
entry2 = tk.Button(window,width=30, text ='Click here for destination directory!!', command = lambda:save_file(),font='Helvetica 8 bold')
entry2.grid(column=1, row=7)    
lb4 = tk.Label(window, text="                                                                                           ")
lb4.grid(column=2, row=7)

entry3 = tk.Button(window,width=30, text ='Generate EDA Report', command = lambda:showMsg(),font='Helvetica 8 bold')
entry3.grid(column=1, row=8) 


#Labels, buttons and entries - Milestone2
llb5 = tk.Label(window, text="")
llb5.grid(column=0, row=19)
lb5 = tk.Label(window, text="Milestone 2 - Text preprocessing and model building",font='Helvetica 8 bold')
lb5.grid(column=1, row=20)    
llb5 = tk.Label(window, text="")
llb5.grid(column=0, row=21)

lb6 = tk.Label(window, text="Step1: Select Source file",font='Helvetica 8 bold')
lb6.grid(column=0, row=22)
entry4 = tk.Button(window,width=30, text ='Click here to load file!!', command = lambda:open_file(),font='Helvetica 8 bold')
entry4.grid(column=1, row=22)    
lb7 = tk.Label(window, text="                                                                                           ")
lb7.grid(column=2, row=22)
entry5 = tk.Button(window,width=30, text ='Generate Vectors', command = lambda:Preprocessing(),font='Helvetica 8 bold')
entry5.grid(column=1, row=23) 

entry6 = tk.Button(window,width=30, text ='Model Status:Undeveloped', command = lambda:Train(),font='Helvetica 8 bold')
entry6.grid(column=1, row=24) 

entry7 = tk.Button(window,width=30, text ='Test(Test Data)', command = lambda:Test(),font='Helvetica 8 bold')
entry7.grid(column=1, row=25) 

lb9 = tk.Label(window, text="                                                                                           ")
lb9.grid(column=1, row=26)
lb8 = tk.Label(window, text="Classification Report : ",font='Helvetica 8 bold')
lb8.grid(column=0, row=27)
t = tk.Label(window, width=30, height=12)
t.grid(column=1, row=28)
lb10 = tk.Label(window, text="                                                                                           ")
lb10.grid(column=1, row=30)
# t = tk.Text(window, width=20, height=3)
    
window.mainloop()